In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
# InvoiceNo: 주문 번호
# StockCode: 상품 코드
# Description: 상품 설명
# Quantity: 상품 수량
# InvoiceDate: 주문 날짜
# UnitPrice: 상품 단가
# CustomerID: 고객 ID
# Country: 국가

In [ ]:
# online_retail.csv 파일 불러오기
df = pd.read_csv(f"data/online_retail.csv")
df = df[df["CustomerID"].notnull()]
df.shape

In [ ]:
df['StockCodeNo'] = df['StockCode'].astype("category").cat.codes
# 상품 코드와 고객 ID로 이루어진 새로운 데이터프레임 생성
new_df = df[['StockCodeNo', 'CustomerID']]

# NaN 값을 가지는 행 제거
new_df = new_df.dropna()

# 고객 ID와 상품 코드를 정수형으로 변환
new_df['CustomerID'] = new_df['CustomerID'].astype(int)
new_df['StockCodeNo'] = new_df['StockCodeNo'].astype(int)

In [ ]:
new_df = df.groupby(["CustomerID", "StockCodeNo"])["Quantity"].sum().reset_index()
new_df.head()

In [ ]:
from sklearn.model_selection import train_test_split

# 데이터셋을 train set과 test set으로 나눔
train_df, test_df = train_test_split(new_df, test_size=0.2)
train_df

In [ ]:
# 사용자 수와 상품 수 계산
n_users = new_df['CustomerID'].nunique()
n_items = new_df['StockCodeNo'].nunique()

# 파이토치에서 사용하기 위해 데이터셋을 텐서로 변환
train_data = torch.FloatTensor(train_df.values)
test_data = torch.FloatTensor(test_df.values)

train_data.shape, test_data.shape, test_data[0]

In [ ]:
train_data = torch.tensor(train_data, dtype=torch.long)
test_data = torch.tensor(test_data, dtype=torch.long)

# 모델 생성
class RecommenderNet(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=50):
        super().__init__()
        self.user_embedding = torch.nn.Embedding(n_users, n_factors)
        self.item_embedding = torch.nn.Embedding(n_items, n_factors)
        self.fc1 = torch.nn.Linear(n_factors*2, 128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.fc3 = torch.nn.Linear(64, 1)
        self.dropout = torch.nn.Dropout(p=0.5)

    def forward(self, x):
        user_ids = x[:, 1].unsqueeze(1)
        item_ids = x[:, 0].unsqueeze(1)

        user_embedding = self.user_embedding(user_ids)
        item_embedding = self.item_embedding(item_ids)

        x = torch.cat([user_embedding, item_embedding], dim=-1)
        x = self.dropout(x)
        x = self.fc1(x)
        x = torch.nn.ReLU()(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = torch.nn.ReLU()(x)
        x = self.dropout(x)
        x = self.fc3(x)

        return x

n_users = train_data[:, 1].max() + 1
n_items = train_data[:, 0].max() + 1

model = RecommenderNet(n_users, n_items, n_factors=50)
# 예측값 계산
print(train_data[0].unsqueeze(0))
prediction = model(train_data[0].unsqueeze(0))
print(prediction.squeeze())

In [ ]:
# 모델 초기화
# model = RecommenderNet(n_users, n_items)

# 옵티마이저와 손실 함수 정의
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

In [ ]:
train_data.shape

In [ ]:
epochs = 10

for epoch in range(epochs):
    train_loss = 0.0
    optimizer.zero_grad()
    outputs = model(train_data)
    loss = criterion(outputs.squeeze(), train_data[:, 2].float())
    loss.backward()
    optimizer.step()
    train_loss += loss.item()

    # train set에서의 RMSE 값을 출력
    train_rmse = np.sqrt(train_loss / len(train_data))
    print("Epoch:", epoch+1, "Train RMSE:", train_rmse)

In [ ]:
# test set에서의 RMSE 값을 계산
test_loss = 0.0
print(test_data.shape)
outputs = model(test_data)
print(outputs.shape)

loss = criterion(outputs.squeeze(), test_data[:, 2].float())
test_loss += loss.item()

test_rmse = np.sqrt(test_loss / len(test_data))
print("Test RMSE:", test_rmse)

In [ ]:
# 추천할 상품의 수 정의
top_k = 10

# 사용자와 아이템 특성 행렬 가져오기
user_embeddings = model.user_embedding.weight.detach().numpy()
item_embeddings = model.item_embedding.weight.detach().numpy()

# 추천 함수 정의
def recommend(user_id):
    # 사용자가 구매한 상품 제외한 상품 중에서 예측 평점이 높은 상품 추출
    user_items = df[df['CustomerID'] == user_id]['StockCode'].unique()
    user_embedding = user_embeddings[user_id-1]
    scores = np.dot(item_embeddings, user_embedding)
    scores = np.array([(i+1, score) for i, score in enumerate(scores)])
    scores = scores[~np.isin(scores[:, 0], user_items)]
    scores = scores[scores[:, 1].argsort()[::-1]]
    top_k_items = scores[:top_k, 0].astype(int)
    return top_k_items

# 예시 사용자에 대해 추천 상품 목록 출력
user_id = 1
recommended_items = recommend(user_id)
print(f"User {user_id}의 추천 상품 목록: {recommended_items}")